# APIs und Authentifikation (Google API)

Wir haben uns vor langer, langer Zeit die Erdbeben-Applikation angeschaut. Die Nutzung ist zwar begrenzt, doch es für deren Nutzung keine weitere Zulassung notwendig. Das ist bei den meisten anderen APIs nicht der Fall. In der Regel verlangen die Besitzer einer API Authentifikation der Nutzer. So können sie kontrollieren, wer wieviel nutzt. Denn auch eine API-Abfrage beansprucht Rechenkraft. Bei Millionen Abfragen kann das schnell ins Geld gehen. Es gibt dabei verschiedene API-Typen: REST, SOAP, XML-RCP oder JSON-RPC. Was die verschiedenen Vorteile dieser API-Technologien sind, schlägt ihr am besten nach. Die gängigsten sind heute REST und SOAP. Die trifft man immer wieder. 

Wie man sich authentifiziert, wollen wir uns nun gemeinsam anschauen. Wir tun das mit der Google Maps API. Zu allererst müssen wir eine Key kreieren. Dafür brauchen wir eine Gmail-Konto und eine Kreditkarte. Wir werden keinen Rappen ausgeben, die erste 100'000 Aufrufe sind gratis. Die Kreditkarte ist trotzdem nötig. Als erstes besuchen wir [also die Google-Maps-Plattform](https://cloud.google.com/maps-platform).

## Die Google API
Die API kann man für verschiedene Sachen verwenden: Um Distanzen zu berechnen, um Standorte mit Geo-Daten auszustatten, oder um Standorte zu suchen. Natürlich nicht einzelne Standorte, sondern ganz viele. In diesem Beispiel werden wir uns bestimmte Standorte suchen.

Nachdem ihr Euch oben bei Google angemeldet habt, müsst ihr eine API Key generieren und abspeichern. Nun gehen wir nur Places API und Places Search. Das [ist hier](https://developers.google.com/places/web-service/intro).

Um Eure API keys zu finden, geht ihr zu API & Services, Credentials. [Hier](https://console.cloud.google.com/apis/credentials?project=clear-canyon-166711)

### Das italienische Restaurant mit den meisten Sternen bei Google Maps?

Wir werden mir der Textsearch-Option arbeiten: ```https://maps.googleapis.com/maps/api/place/textsearch/output?parameters```. Wir geben ein:
- https://maps.googleapis.com/maps/api/place/textsearch/json
- query=italienische+restaurants+in+zurich
- fields=formatted_address,name,rating,opening_hours,pagetoken
- key=XXXXXX

In [1]:
#Hier kreiere ich die URLs, um die API abzufragen.
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
query = "query=italienische+restaurants+in+zurich&"
fields = "fields=formatted_address,name,rating,opening_hours,pagetoken&"
locationbias = "locationbias=point:lat,lng&"
key = "key=XXXXXXXXXX"

In [ ]:
api_query = url+query+fields+locationbias+key

In [ ]:
api_query

importieren wir requests

In [ ]:
import requests

In [ ]:
r = requests.get(api_query)
result = r.json()

In [ ]:
len(result)

In [ ]:
for key in result:
    print(key)

In [ ]:
result['results'][0]['geometry']['location']

In [ ]:
result['results'][0]['name']

In [ ]:
len(result['results'])

In [ ]:
new_lst = []
#Jetzt gehen wir Schritt für Schritt durch die Treffer, es sind pro Abfrage 20.

for beiz in result['results']:
    address = beiz['formatted_address']
    name = beiz['name']
    rating = beiz['rating']
    lat = beiz['geometry']['location']['lat']
    long = beiz['geometry']['location']['lng']
    
    mini_dict = {'Addresse': address, 
                 'Name': name,
                 'Rating': rating,
                 'Lat':lat,
                 'Lng':long}
    
    new_lst.append(mini_dict)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(new_lst)

Aber, das sind nur 20. Wir wollen mehr. Dafür müssen wir mit diesem Nexttoken arbeiten. 

In [ ]:
import time

In [ ]:
new_list = []

#Mit der Pagetoken, die wir zuerst auf "" also leer setzen, können wir 
#der URL sagen, dass Sie immer 20 Treffer ab einem bestimmten Index
#anzeigen soll. 

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
query = "query=italienische+restaurants+in+zurich&"
fields = "fields=formatted_address,name,rating,opening_hours,pagetoken&"
locationbias = "locationbias=point:lat,lng&"
key = "key=XXXXXXXXXX&"
pagetoken = ''

result = range(4) #Hier definieren wir die Länge von Result. 
while len(result) > 3: #Die While Schlaufe (wie ein For Loop, aber mit Bedingung),
                       #wird nur dann ausgeführt, wenn result grösser als drei sit
                       #das ist beim ersten Mal immer der Fall, weil wir haben 
                       #Result ja eben gesetzt.
    
    print(len(result)) #Wir drucken result nun einfach aus. Beim ersten MAl wird
                       #Result natürlich 4 sein. 

    api_query = url+query+fields+locationbias+key+pagetoken
    print(api_query)
    time.sleep(2)
    r = requests.get(api_query)
    result = r.json()  #Hier wird result nun neu definiert, es ist nun nicht mehr 
                       #nur die Zahl 4, sondern das Ergebnis der Abfrage.
    
    for elem in result['results']:
        address = elem['formatted_address']
        name = elem['name']
        rating = elem['rating']
        lat = elem['geometry']['location']['lat']
        long = elem['geometry']['location']['lng']
    
        mini_dict = {'Addresse': address, 
                     'Name': name,
                     'Rating': rating,
                     'Lat':lat,
                     'Lng':long}
    
        new_list.append(mini_dict)
        
    
    if len(result) > 3: #Nun prüfen wir die Länge von Result. Wenn sie weniger als 4 ist
                        #heisst das, dass es keinen Pagetoken gibt. Folglich gibt es auch keine
                        #Ergebnisse mehr. In diesem Fall springt der Code zu else. Und bricht ab.
        pagetoken = 'pagetoken='+result['next_page_token'] 
    else:
        break

In [ ]:
pd.DataFrame(new_list)

Das ist leider nicht zufriedenstellend, denn offenbar begrenzt sich Google selber. Wir müssen die nexttoken Suche mit einer Suche nach Städten erweitern. Suchen wir "schweizer städte .csv" bei Google.

## Ergänzung Schweizer Städte

In [ ]:
t = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_St%C3%A4dte_in_der_Schweiz#St%C3%A4dte_mit_mindestens_10'000_Einwohnern")

In [ ]:
def citynames(elem):
    elem = elem.replace(" 1","").replace(" 2", "").replace(" / 3","")
    return elem
cities = list(t[0][1:][1].apply(citynames)) #nun haben wir eine Liste aller Orte in der Schweiz mit aktuell
                                            #10'000 und mehr Einwohner und Einwohnerinnen.

## Jetzt ergänzen wir diese Städte 

In [ ]:
new_list = []

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
query = "query=italienische+restaurants+in+"
fields = "fields=formatted_address,name,rating,opening_hours,price_level,pagetoken&"
locationbias = "locationbias=point:lat,lng&"
key = "key=XXXXXXXX&"
pagetoken = ''

for city in cities[:10]: #Nur die ersten Städte
    result = range(4)
    while len(result) > 3: 
    
        print(len(result))
        api_query = url+query+city+"&"+fields+locationbias+key+pagetoken
        print(api_query)
        time.sleep(2)
        r = requests.get(api_query)
        result = r.json()
        print(len(result))
    
        for elem in result['results']:
            address = elem['formatted_address']
            name = elem['name']
            rating = elem['rating']
            lat = elem['geometry']['location']['lat']
            long = elem['geometry']['location']['lng']
    
            mini_dict = {'Addresse': address, 
                     'Name': name,
                     'Rating': rating,
                     'Lat':lat,
                     'Lng':long}
    
            new_list.append(mini_dict)
        
    
        if len(result) > 3:
            pagetoken = 'pagetoken='+result['next_page_token'] 
        else:
            pagetoken = ''
            break



In [ ]:
df = pd.DataFrame(new_list)

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.to_csv('italrest_allcities.csv')